In [258]:
import datetime
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
from palettable import colorbrewer
import matplotlib as mpl
import matplotlib.colors as colors
import cartopy.crs as ccrs
from cartopy.examples.arrows import sample_data
#for movie
import holoviews as hv



In [259]:
#climatology
dir_data = 'F:/data/sat_data/oscar/L4/oscar_third_deg/'
filename = dir_data + 'climatology_1993_2018_monthly_data_oscar.nc'
ds=xr.open_dataset(filename)
ds.close()
ds = ds.sel(lon=slice(110,250),lat=slice(65,5))



In [ ]:
#put 5 day data into one big array or next section is for monthly (smaller)
dir_data = 'F:/data/sat_data/oscar/L4/oscar_third_deg/'
for lyr in range(1993,2018): #2019): #2017):
    filename = dir_data + 'oscar_vel' + str(lyr).zfill(4) + '.nc'
    ds=xr.open_dataset(filename,drop_variables=['um','vm'])
    ds = ds.rename({'longitude':'lon','latitude':'lat'})
    ds = ds.drop('year')
    ds['spd']=np.sqrt(ds.u*ds.u+ds.v*ds.v)
    ds_subset = ds.sel(longitude=slice(110,250),latitude=slice(65,0))
    ds.close()
    if lyr==1993:
        ds_all = ds_subset
    else:
        ds_all=xr.concat([ds_all,ds_subset],'time')
   

In [260]:
#put monthly data into one big array
dir_data = 'F:/data/sat_data/oscar/L4/oscar_third_deg/'
for lyr in range(1993,2018): #2019): #2017):
    filename = dir_data + str(lyr) + 'monthly_data_oscar.nc'
    ds=xr.open_dataset(filename)
    ds_subset = ds.sel(lon=slice(120,250),lat=slice(65,15))
    ds.close()
    if lyr==1993:
        ds_all = ds_subset
    else:
        ds_all=xr.concat([ds_all,ds_subset],'time')


In [261]:
ds_all

<xarray.Dataset>
Dimensions:  (depth: 1, lat: 151, lon: 391, time: 300, year: 1800)
Coordinates:
  * year     (year) float64 1.993e+03 1.993e+03 ... 2.018e+03 2.018e+03
  * depth    (depth) float32 15.0
  * lat      (lat) float64 65.0 64.67 64.33 64.0 63.67 ... 16.0 15.67 15.33 15.0
  * lon      (lon) float64 120.0 120.3 120.7 121.0 ... 249.0 249.3 249.7 250.0
  * time     (time) datetime64[ns] 1993-01-31 1993-02-28 ... 2017-12-31
Data variables:
    u        (time, depth, lat, lon) float64 nan nan nan ... -0.01118 -0.04393
    v        (time, depth, lat, lon) float64 nan nan nan ... -0.02956 0.0342
    spd      (time, depth, lat, lon) float64 nan nan nan ... 0.05109 0.08189
    dir      (time, depth, lat, lon) float64 nan nan nan ... -60.87 -52.98 69.87

In [262]:
ds_subset = ds_all.sel(time>dt.datetime(2000,1,1))


TypeError: '>' not supported between instances of 'type' and 'datetime.datetime'

In [ ]:
fig, axarr = plt.subplots(1,1,figsize=(4,4))
ds_all.spd[0,0,:,:].plot(vmin=0,vmax=.3)
fig.savefig('F:/data/NASA_biophysical/telecon_figs/' + 'map.png', dpi=100)

In [ ]:
fig, axarr = plt.subplots(1,1,figsize=(4,4))
ds_subset = ds_all.sel(lon=slice(180,225),lat=slice(65,40))
ds_subset.spd[120,0,:,:].plot(vmin=0,vmax=.3)


In [ ]:
min_lat = np.empty(250)
max_lat = np.empty(250)
#max_lat[185:]=[52.0,52.1,52.1,52.1,54]
for cen_lon in range(185,195):
    fig, axarr = plt.subplots(1,1,figsize=(4,4))
    ds_subset = ds_all.sel(lon=slice(cen_lon,cen_lon),lat=slice(60,50))
    ds_subset.spd[84:,0,:,:].transpose().plot(vmin=0,vmax=.3)
    fig.savefig('F:/data/NASA_biophysical/telecon_figs/' + 'hov_lon'+str(cen_lon)+'.png', dpi=100)


In [ ]:
cen_lon=185
ds_subset = ds_all.sel(lon=slice(cen_lon,cen_lon),lat=slice(60,50))
tem = ds_subset.mean('time').mean('depth').drop('year')
print(tem)
ii = tem.argmax(dim='lat')
ilat = tem.lat[ii.spd].data[0]
print(ilat)

In [ ]:
cen_lon=185
import datetime as dt
svlat=[]
svspd=[]
ilon=195 #for ilon in range(195,285):
cen_lon = ds_all.lon[ilon]
#for this lon find the mean position
ds_subset = ds_all.sel(lon=slice(cen_lon,cen_lon),lat=slice(60,48))
tem = ds_subset.mean('time').mean('depth').drop('year')
ii = tem.argmax(dim='lat')
cen_lat = tem.lat[ii.spd].data[0]
print(cen_lat)
#now look around cen lat for tim variable mean 
ds_subset = ds_all.sel(time=slice(dt.datetime(2000,1,1),dt.datetime(2018,1,1)),lon=slice(cen_lon,cen_lon),lat=slice(cen_lat+2,cen_lat-4))
tem = ds_subset.mean('depth').drop('year')
ii = tem.argmax(dim='lat')
yy = tem.max(dim='lat')
cen_ii_time=ii.spd
cen_lat_time=tem.lat[ii.spd]
cen_lat_spd=yy.spd
print(cen_lat_time.mean())
cen_lat_time.plot()
svlat.append(cen_lat_time)
svspd.append(cen_lat_spd)



In [ ]:
tem

In [ ]:
tem.spd[:,:,0].plot()

In [ ]:
cen_lat_time.plot()

In [ ]:
ds_all.spd[10,0,:,:].plot()
print(ds_all.lon[195])

In [ ]:

ilon,itime=195,180 #for ilon in range(195,285):
cen_lon = ds_all.lon[ilon]
ds_subset = ds_all.sel(lon=slice(cen_lon,cen_lon),lat=slice(60,47.3))
ds_all.spd[itime,0,:,ilon].plot()
ds_subset.spd[itime,0,:,0].plot()
ds_smooth = ds_subset.rolling(time=4).mean()
ds_smooth.spd[itime,0,:,0].plot()  #smooth data
tem = ds_subset.mean('time').mean('depth').drop('year')
ii = tem.argmax(dim='lat')
cen_lat = tem.lat[ii.spd].data[0]
cen_spd = tem.spd[ii.spd].data[0]
plt.plot(cen_lat,cen_spd,'b.')
print(cen_lat,cen_spd)
#now look around cen lat for tim variable mean 
ds_subset = ds_all.sel(time=slice(dt.datetime(2000,1,1),dt.datetime(2018,1,1)),lon=slice(cen_lon,cen_lon),lat=slice(cen_lat+2,cen_lat-4))
tem = ds_subset.mean('depth').drop('year')
ii = tem.argmax(dim='lat')
yy = tem.max(dim='lat')
cen_ii_time=ii.spd
cen_lat_time=tem.lat[ii.spd]
cen_lat_spd=yy.spd
#print(cen_lat_time)
print(cen_lat_time.mean())
#cen_lat_time.plot()
plt.plot(cen_lat_time.mean(),cen_lat_spd.mean(),'r.')

In [ ]:
cen_lat_time[10,0]

In [ ]:
ilon=195
cen_lon = ds_all.lon[ilon]
#for this lon find the mean position
ds_subset = ds_all.sel(lon=slice(cen_lon,cen_lon),lat=slice(60,48))
ds_subset.spd[86,0,:,0].plot()

In [ ]:
ds_subset.spd[0,:]

In [ ]:
fig, axarr = plt.subplots(1,1,figsize=(4,4))
ds_subset = ds_all.sel(lon=slice(180,180),lat=slice(65,45))
ds_subset.spd[:,0,:,:].transpose().plot(vmin=0,vmax=.3)
fig.savefig('F:/data/NASA_biophysical/telecon_figs/' + 'hov_lon180.png', dpi=100)


In [ ]:
fig, axarr = plt.subplots(1,1,figsize=(4,4))
ds_subset = ds_all.sel(lon=slice(190,190),lat=slice(65,45))
ds_subset.spd[:,0,:,:].transpose().plot(vmin=0,vmax=.3)
fig.savefig('F:/data/NASA_biophysical/telecon_figs/' + 'hov_lon190.png', dpi=100)



In [ ]:
fig, axarr = plt.subplots(1,1,figsize=(4,4))
ds_subset = ds_all.sel(lon=slice(200,200),lat=slice(65,45))
ds_subset.spd[:,0,:,:].transpose().plot(vmin=0,vmax=.3)
fig.savefig('F:/data/NASA_biophysical/telecon_figs/' + 'hov_lon200.png', dpi=100)


In [ ]:
fig, axarr = plt.subplots(1,1,figsize=(4,4))
ds_subset = ds_all.sel(lon=slice(210,210),lat=slice(65,45))
ds_subset.spd[:,0,:,:].transpose().plot(vmin=0,vmax=.3)
fig.savefig('F:/data/NASA_biophysical/telecon_figs/' + 'hov_lon210.png', dpi=100)


In [ ]:
fig, axarr = plt.subplots(1,1,figsize=(4,4))
ds_subset = ds_all.sel(lon=slice(180,210),lat=slice(58,47.5))
ds_subset.spd[:,0,:,:].max('lat').transpose().plot(vmin=0,vmax=0.5)
fig.savefig('F:/data/NASA_biophysical/telecon_figs/' + 'max_spd_180_210.png', dpi=100)


In [ ]:
fig, axarr = plt.subplots(1,1,figsize=(4,4))
ds_subset = ds_all.sel(lon=slice(220,245),lat=slice(42,42))
ds_subset.spd[:,0,:,:].transpose().plot(vmin=0,vmax=.15)
fig.savefig('F:/data/NASA_biophysical/telecon_figs/' + 'hov_lat42.png', dpi=100)


In [ ]:
ds_all

In [ ]:
import cartopy.crs as ccrs
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=-180))
ds_all.spd[0,0,:,:].plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(),vmin=-1,vmax=1,cmap='seismic');
ax.set_global(); ax.coastlines();
ax.set_xlim(-80,70)
ax.set_ylim(15,65)
fig.savefig('F:/data/NASA_biophysical/telecon_figs/' + 'sample_plot.png', dpi=100)


In [ ]:
#try to make movie
ds_all
hv.extension('bokeh')


In [ ]:
ds_all

In [ ]:
%%output holomap='scrubber'
%%opts Image style(cmap='rainbow') plot[colorbar=True]
%%opts Image [width=500, height=400]
opts = dict(cmap='rainbow', colorbar=True, width=300, height=230, axiswise=True)
hv_ds = hv.Dataset(ds_all.spd[1:150,0,:,:])
img = hv_ds.to(hv.Image, ['lon', 'lat']).options(**opts)
#img.options(colorbar=True, width=300, height=230, axiswise=True)
img.redim.range(spd=(0,.5))


In [ ]:
hv.save(img, 'Spd_half_A.html')

In [ ]:
%%output holomap='scrubber'
%%opts Image style(cmap='viridis') plot[colorbar=True]
%%opts Image [width=500, height=400]
opts = dict(cmap='rainbow', colorbar=True, width=300, height=230, axiswise=True)
hv_ds = hv.Dataset(ds_all.spd[100:300,0,:,:])
img = hv_ds.to(hv.Image, ['lon', 'lat']).options(**opts)
#img.options(colorbar=True, width=300, height=230, axiswise=True)
img.redim.range(spd=(0,.5))


In [ ]:
hv.save(img, 'Spd_half_B.html', backend='bokeh')

In [ ]:
hv.extension('matplotlib')

air = xr.tutorial.load_dataset('air_temperature')
ds = hv.Dataset(air.isel(time=range(100)))
images = ds.to(hv.Image, ['lon', 'lat']).options(fig_inches=(10, 5), colorbar=True, cmap='viridis')


In [ ]:
#Y, X = np.meshgrid(ds.latitude, ds.longitude)
#ds = ds.sel(longitude=slice(110,250),latitude=slice(65,15))
import matplotlib as mpl
import matplotlib.colors as colors
for imon in range (5,6): #13):
    #ds = ds.sel(lon=slice(110,250),lat=slice(65,15))
    X = ds.lon.values
    Y = ds.lat.values
    U = ds.u[imon,0,:,:]
    V = ds.v[imon,0,:,:]
    speed = np.sqrt(U*U + V*V)
    speed = speed.fillna(0)
    lws = 10*speed / speed.max()

    fig0, ax0 = plt.subplots()
    strm = ax0.streamplot(X,Y, U, V, density=6,color=V.values,linewidth=lws.values, cmap='seismic',norm=colors.Normalize(vmin=-.5,vmax=.5))
    fig0.colorbar(strm.lines)


In [ ]:
    ds2 = ds.sel(lon=slice(110,179),lat=slice(65,5))
    lons1 = ds2.lon.values
    lons1[lons1 > 180] = lons1[lons1 > 180] - 360.  #wrap -180-180
    lats1 = ds2.lat.values
    
#    lons0, lats0 = np.meshgrid(lons1,lats1)
    X = lons1 
    Y = lats1
    U = ds2.u[imon,0,:,:]
    V = ds2.v[imon,0,:,:]
    speed = np.sqrt(U*U + V*V)
    speed = speed.fillna(0)
    lws = 10*speed / speed.max()
    
    Xg, Yg = np.meshgrid(X,Y)
    input_projection = ccrs.PlateCarree()
    #ax = plt.subplot(projection=output_projection)
    #ax.streamplot(X, Y, U, V, transform=input_projection)
#    ax0 = plt.axes(projection=ccrs.PlateCarree())
#    ax0 = fig0.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=180.0))
#    ax0.set_extent([-110, 110, 5, 65], crs=ccrs.PlateCarree(central_longitude=180.0))
#    ax0.coastlines()

#    fig0, ax0 = plt.subplots()

    #fig0, ax0 = plt.subplots()
    #ax0 = plt.axes(projection=ccrs.PlateCarree())
    ax0 = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
    strm = ax0.streamplot(X,Y, U, V, density=6,color=V.values,linewidth=lws.values, cmap='seismic',norm=colors.Normalize(vmin=-.5,vmax=.5), transform = input_projection)
    fig0.colorbar(strm.lines)

    #fig0, ax0 = plt.figure(figsize=(10, 5))
    #strm = ax0.streamplot(X,Y, U, V, density=6,color=V.values,linewidth=lws.values, cmap='seismic',
    #                      norm=colors.Normalize(vmin=-.5,vmax=.5)) #, transform=input_projection)
#    ax0.coastlines()
#    fig0.colorbar(strm.lines)


In [ ]:
X

In [ ]:
#Y, X = np.meshgrid(ds.latitude, ds.longitude)
#ds = ds.sel(longitude=slice(110,250),latitude=slice(65,15))

for imon in range (5,6): #13):
    #ds = ds.sel(lon=slice(110,250),lat=slice(65,15))
    lons1 = ds.lon.values
    lons1[lons1 > 180] = lons1[lons1 > 180] - 360.  #wrap -180-180
    lats1 = ds.lat.values
    
#    lons0, lats0 = np.meshgrid(lons1,lats1)
    X = lons1 
    Y = lats1
    U = ds.u[imon,0,:,:]
    V = ds.v[imon,0,:,:]
    Vnorm = V/abs(V).max()
    speed = np.sqrt(U*U + V*V)
    speed = speed.fillna(0)
    lws = 10*speed / speed.max()
    fig = plt.figure(figsize=(8,8))
#    map = Basemap(projection='cyl',llcrnrlat=lats1[-1],llcrnrlon=lons1[0],
#                  urcrnrlat=lats1[0],urcrnrlon=lons1[-1],resolution='l')

    map = Basemap(projection='lcc', llcrnrlon=110, llcrnrlat=5, 
                  urcrnrlon=-110, urcrnrlat=65,lat_0=5, lon_0=-98.)
    map.drawcoastlines(linewidth=0.5)
    strm = map.streamplot(X,Y, U, V, latlon=True, density=6,color=V.values,linewidth=lws.values, cmap='seismic',norm=colors.Normalize(vmin=-.5,vmax=.5))
    #norm = mpl.colors.Normalize(vmin=-.5,vmax=.5)
    #fig0.colorbar(strm.lines)


In [ ]:
from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt
# llcrnrlat,llcrnrlon,urcrnrlat,urcrnrlon
# are the lat/lon values of the lower left and upper right corners
# of the map.
# resolution = 'c' means use crude resolution coastlines.
m = Basemap(projection='cyl',llcrnrlat=-90,urcrnrlat=90,\
            llcrnrlon=110,urcrnrlon=-110,resolution='c')
m.drawcoastlines()
m.fillcontinents(color='coral',lake_color='aqua')
# draw parallels and meridians.
m.drawparallels(np.arange(-90.,91.,30.))
m.drawmeridians(np.arange(-180.,181.,60.))
m.drawmapboundary(fill_color='aqua')
plt.title("Equidistant Cylindrical Projection")
plt.show()

In [ ]:
print(Xg.shape,xx.shape,yy.shape,U.shape,V.shape)

In [ ]:
lons = ds.lon.values
lons[lons > 180] = lons[lons > 180] - 360.
lons

In [ ]:
print(X.shape,Y.shape,U.shape,V.shape)

In [ ]:
import matplotlib.animation as animation
X = ds.lon.values
Y = ds.lat.values
U = ds.u[:,0,:,:]
V = ds.v[:,0,:,:]
Vnorm = V/abs(V).max()

fig, ax = plt.subplots()
#data = np.random.rand(2, 25)
l, = ax0.streamplot(X,Y, U, V, density=6,color=Vnorm.values, linewidth=lws.values, cmap='seismic')
line_ani = animation.FuncAnimation(fig, update_line, 25, fargs=(data, l),
                                   interval=50, blit=True)


In [ ]:
print(V.min(),V.max())

In [ ]:

cmap = colorbrewer.get_map('Spectral', 'diverging', 11, reverse=True).mpl_colormap
fig = plt.figure(figsize=(11.7,8.3))
m = Basemap(projection='merc', lat_0 = 40, lon_0 = -179, resolution = 'l', area_thresh = 0.1,
            llcrnrlon=-110.0, llcrnrlat=25.0,urcrnrlon=-114.0, urcrnrlat=38.)
m.bluemarble()
lat_grid, lon_grid = np.meshgrid(ds.lat.values, ds.lon.values)
x,y = m(lon_grid,lat_grid)
cs = m.pcolormesh(x,y, speed.transpose(), cmap=cmap, vmin = 0, vmax = 5)


In [ ]:
import matplotlib.pyplot as plt
# setup Lambert Conformal basemap.
# set resolution=None to skip processing of boundary datasets.
m = Basemap(width=12000000,height=9000000,projection='lcc',
            resolution=None,lat_1=2.,lat_2=65,lat_0=50,lon_0=-110.)
m.bluemarble(ax=None, scale=0.25)
plt.show()

In [ ]:
import matplotlib
print(matplotlib.__version__)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

lats1=[-25, 66]
lons1=[60, 179.95]
fig=plt.figure()
fig.set_size_inches((10, 9))

ax1=fig.add_subplot(121)
ax2=fig.add_subplot(122)

bmap=Basemap(projection='merc',\
        llcrnrlat=lats1[0], llcrnrlon=lons1[0],\
        urcrnrlat=lats1[1], urcrnrlon=lons1[1],\
        lon_0=(lons1[0]+lons1[1])/2.,\
        ax=ax1)

bmap.bluemarble(ax = ax1)

ax1.set_position([0, 0, 0.8, 1])

lats2=[-25,66]
lons2=[-179.95, -150]
bmap=Basemap(projection='merc',\
        llcrnrlat=lats2[0], llcrnrlon=lons2[0],\
        urcrnrlat=lats2[1], urcrnrlon=lons2[1],\
        lon_0=(lons2[0]+lons2[1])/2.,\
        ax=ax2)

bmap.bluemarble(ax = ax2)
ax2.set_position([0.8, 0, 0.2, 1])

ax1.set_aspect('auto')
ax2.set_aspect('auto')

ax1.set_adjustable('datalim')
ax2.set_adjustable('datalim')
ax1.axis('off')
ax2.axis('off')
plt.show()


In [ ]:
ds.u[0,0,:,:].plot()
ds.u[0,0,0,0]

In [ ]:
ds

In [ ]:
#make monthly climatology, need to drop 'year' coordinate value for mean to work right
for lyr in range(1993,2019):
    filename = dir_data + str(lyr) + 'monthly_data_oscar.nc'
    ds=xr.open_dataset(filename)
    ds2 = ds.drop('year')
    if lyr==1993:
        ds_clim = ds2
    else:
        ds_clim = xr.concat([ds_clim,ds2],dim = 'time')
    ds.close()
ds_clim2 = ds_clim.groupby('time.month').mean('time')
ds_clim2.to_netcdf(dir_data + 'climatology_1993_2018_monthly_data_oscar.nc')


In [ ]:
ds

In [ ]:
#make monthly climatology, need to drop 'year' coordinate value for mean to work right
for lyr in range(2000,2019):
    filename = dir_data + str(lyr) + 'monthly_data_oscar.nc'
    ds=xr.open_dataset(filename)
    ds2 = ds.drop('year')
    if lyr==2000:
        ds_clim = ds2
    else:
        ds_clim = xr.concat([ds_clim,ds2],dim = 'time')
    ds.close()
ds_clim2 = ds_clim.groupby('time.month').mean('time')
ds_clim2.to_netcdf(dir_data + 'climatology_2000_2018_monthly_data_oscar.nc')

In [ ]:
#OLD OLD OLD OLD OLD OLD CODE
#make oscar climatology, OLD version of code
#dir_data='F:/data/sat_data/oscar/L4/oscar_third_deg/'
dir_clim='F:/data/sat_data/oscar/L4/oscar_third_deg/climatology/'
dir_data = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/oscar/preview/L4/oscar_third_deg/'
for lyr in range(1993,2018):
    filename = dir_data + 'oscar_vel' + str(lyr).zfill(4) + '.nc.gz'
    ds=xr.open_dataset(filename,drop_variables=['um','vm'])
    ds_count=data_ones(ds)
    #in order to add up the data the time arrays have to be aligned otherwise xarray doesn't know what to do with it
    ds['time'] = ds['time'] - np.datetime64(lyr,'Y')
    ds_count['time'] = ds_count['time'] - np.datetime64(lyr,'Y')
    if lyr==1993:
        ds2=ds.fillna(0)
        ds_count2=ds_count.fillna(0)
        ds_summer=ds2
        ds_counter = ds_count2
    else:
        ds2 = ds.interp_like(ds_summer.time)
        ds_count2 = ds_count.interp_like(ds_summer.time)
        ds2=ds2.fillna(0)
        ds_count2=ds_count2.fillna(0)
        ds_summer=ds_summer + ds2
        ds_counter = ds_counter + ds_count2
    print(lyr)
    print(ds.u.shape)
    print(ds_summer.u.shape)
   # print(ds_counter.u.shape)
   # print(ds.u.shape)
   # print(ds.time.data[0:20])
ds_ave = ds_summer / ds_counter
ds_ave.to_netcdf(dir_clim + 'oscar_v2009_1993_2016_climatology_12202118.nc')

In [ ]:
import datetime as dt
tem_date=dt.datetime(2016,1,15)
print(tem_date)
filename = 'https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/godas/dbss_obml.2016.nc'
#filename = 'F:/data/model_data/godas/dbss_obml.2017.nc'
ds = xr.open_dataset(filename)
ds_storm = ds.interp(time = tem_date)
ds_storm


In [ ]:
            dir_ccmp='F:/data/sat_data/ccmp/v02.0/Y'
            storm_date = dt.datetime(2015,1,1)
            syr, smon, sdym, sjdy=str(storm_date.year),str(storm_date.month),str(storm_date.day),str(storm_date.timetuple().tm_yday)
            fname_tem='/CCMP_Wind_Analysis_' + syr + smon.zfill(2) + sdym.zfill(2) + '_V02.0_L3.0_RSS.nc'
            ccmp_filename = dir_ccmp + syr + '/M' + smon.zfill(2) + fname_tem      
            ds=xr.open_dataset(ccmp_filename,drop_variables=['nobs'])
            ds

In [ ]:
ds_storm.dbss_obml.plot(vmin=0,vmax=200)

In [ ]:
ds_storm.dbss_obml[0,0]